# Edge Privacy
Reference implementations of techniques described in the paper [Smooth Sensitivity and Sampling in Private Data Analysis](https://cs-people.bu.edu/ads22/pubs/NRS07/NRS07-full-draft-v1.pdf) by Kobbi Nissim, Sofya Raskhodnikova, and Adam Smith.

## Library Imports
We use [networkx](https://networkx.org) to perform the required graph computations and the implementation of the Cauchy mechanisms provided by [RelM](https://github.com/anusii/RelM) to release the differentially private query responses.

In [1]:
import numpy as np
import networkx as nx
from relm.mechanisms import CauchyMechanism

<hr style="border:2px solid gray"> </hr>

## Triangle Counts
We construct a differentially private release mechanism for the number of triangles in a graph. This process is comprised of three steps:
  1. Compute the exact query response,
  2. Compute the smooth sensitivity of the query,
  3. Add noise scaled according to the smooth sensitvity.

### Compute the exact query response

#### Generate a random graph

In [2]:
n = 2**13
p = 0.01
g = nx.random_graphs.gnp_random_graph(n, p)

#### Compute the exact triangle count

In [3]:
# Notice here that we divide by three becuase nx.triangles returns the
# sum of the number of triangles each vertex is in.  Therefore, each triangle
# gets counted three times, once for each of its vertices.
triangle_count = np.array([sum(nx.triangles(g).values()) / 3.0])

### Compute the smooth sensitivity of the query

#### Compute the partial count matrices
The algorithm that we use to compute the smooth sensitivity of the query is given in terms of the adjacency matrix $X = \{x_{ij}\}$ where $x_{ij} = \mathbf{1}((i,j) \in E)$. Let the matrix $A$ count the number of triangles that involve each potential edge.  That is, $A = \{a_{ij}\}$ where $a_{ij} = \sum_{k \in [n]} x_{ik} \cdot x_{kj}$. Let the matrix $B$ count the number of half-built triangles that involve each potential edge. That is $B = \{b_{ij}\}$ where $b_{ij} = \sum_{k \in [n]} x_{ik} \oplus x_{kj}$.

In [4]:
# Compute the adjacency matrix
X = nx.linalg.graphmatrix.adjacency_matrix(g).astype(np.int)

# Compute A and B using matrix operations
A = (X @ X).todense()
B = X.sum(0) + X.sum(1) - 2 * A

# Zero out the main diagonal because we are
# interested only in indices (i,j) with i != j
np.fill_diagonal(A, 0)
np.fill_diagonal(B, 0)

#### Compute the local sensitivity at distance $s$ for $0 \leq s \leq n$
We recall from the paper that if $S_{f,\epsilon}^*(G)$ is the smooth sensitivity of a query at an input $G$, $LS_f(H)$ is the local sensitivity of $f$ at an input $H$, and $$A^{(s)}(G) = \max_{H: d(G,H) \leq s} LS_f(H)$$
is the sensitivity of $f$ at distance $s$, then we have
$$S_{f,\epsilon}^*(G) = \max_{0 \leq s \leq n} e^{-\epsilon s} A^{(s)}(G).$$

Furthermore, Nissim et al show that if $f$ counts the number of triangles in a graph then we have
$$A^{(s)}(G) = \max_{0 \leq i \neq j \leq n} c_{ij}(s) \quad \text{where} \quad c_{ij} = \min \left(a_{ij} + \left\lfloor\frac{s + \min(s,b_{ij})}{2}\right\rfloor, n-2 \right).$$

They then describe an $O(n^2 \log n)$ algorithm for computing $A^{(s)}(G)$ for $0 \leq s \leq n$ which works by efficiently identifying the pairs $(a_{ij}, b_{ij})$ needed to compute $c_{ij}(s)$ for all $s$. 

In [5]:
def find_maximal_pairs(A, B):
    """
    Find maximal pairs A[i,j], B[i,j].
    
    Parameters:
        A: A matrix describing the number of potential triangles involving each potential edge (i,j)
        B: A matrix describing the number of half-built triangles involving each potential edge (i,j)
        
    Returns:
        A generator of pairs (a, b) where for for each distinct value a in set(A)
        b = max(B[i,j]) where the maximum is taken over all indices {i,j} where A[i,j] = a.
    """
    W = np.array(A).flatten()
    X = np.array(B).flatten()
    idx = np.argsort(W)
    Y = W[idx]
    Z = X[idx]
    delta = np.concatenate((np.zeros(1), Y[1:] != Y[:-1]))
    new_val_idxs = np.concatenate((np.array([0]),
                                   np.where(delta)[0],
                                   np.array([len(Y)])))
    S = np.empty((len(new_val_idxs)-1), dtype=np.int)
    T = np.empty((len(new_val_idxs)-1), dtype=np.int)
    for i in range(len(new_val_idxs)-1):
        S[i] = Y[new_val_idxs[i]]
        T[i] = np.max(Z[new_val_idxs[i]:new_val_idxs[i+1]])
    return zip(S[::-1], T[::-1])

In [6]:
def find_survivors(maximal_pairs, n):
    """
    Find (i,j) used to compute local sensitivity at distance 0 <= s <= n.
    
    First identify all maximal pairs (A[i,j], B[i,j]).
    Then find the maximal pairs that will produce the correct value for the
    local sensitivity at distance s computations.
    
    Parameters:
        maximal_pairs: A generator of maximal pairs as returned by find_maximal_pairs
        n: The number of nodes in the underlying graph
        
    Returns:
        A sorted list of survivors that can be used to compute the local sensitivity
        at each distance 0 <= s <= n.
    """
    prev_survivor = next(maximal_pairs)
    break_points = {prev_survivor: n+1}
    for survivor in maximal_pairs:
        a0, b0 = prev_survivor
        a1, b1 = survivor
        intersection = 2*(a0 - a1) + b0
        if b0 <= intersection <= b1:
            break_points[prev_survivor] = intersection
            break_points[survivor] = n+1
            prev_survivor = survivor
    survivors = sorted(break_points.items(), key=lambda _: _[1])
    return survivors

In [7]:
def local_sensitivity_at_distance(s, break_list):
    """
    Compute the local sensitivity at distance s.
    
    Parameters:
        s: The distance at which to compute the local sensitivity.
        break_list: A dictionary of candidate (key, value) pairs that
                    are used to compute local sensitivities at various distances.
                    
    Returns:
        The local sensitivity at distance s.
    """
    a, b = next((k for k,v in break_list if s <= v))
    return np.minimum(a + np.floor((s + np.minimum(s, b)) / 2.0), n - 2)

In [8]:
# Compute the local sensitivity at distance s for 0 <= s <= n
maximal_pairs = find_maximal_pairs(A, B)
survivors = find_survivors(maximal_pairs, n)
lsd = np.array([local_sensitivity_at_distance(s, survivors) for s in range(n+1)])

# Compute the smooth sensitivity
epsilon = 1.0
smooth_scaling = np.exp(-epsilon * np.arange(n + 1))
smooth_sensitivity = np.max(lsd * smooth_scaling)

### Add noise scaled according to the smooth sensitvity
We create a differentially private release mechanism by adding noise from the Cauchy distribution scaled according to the smooth sensitivity. Because the Cauchy distributed random variables are real-valued, the differentially private query response will be real-valued despite the exact query response being integer-valued.

In [9]:
# Create a differentially private release mechanism
mechanism = CauchyMechanism(epsilon=epsilon)

# Compute the differentially private query response
dp_triangle_count = mechanism.release(triangle_count, smooth_sensitivity)

#### Display results

In [10]:
print("Exact triangle count = %i" % int(triangle_count))
print("Differentially private triangle count = %f" % dp_triangle_count)

Exact triangle count = 92040
Differentially private triangle count = 92041.455515


<hr style="border:2px solid gray"> </hr>

## Cost of a Minimum Spanning Tree
We construct a differentially private release mechanism for the cost of a minimum spanning tree. As before, this process is comprised of three steps:
  1. Compute the exact query response,
  2. Compute the smooth sensitivity of the query,
  3. Add noise scaled according to the smooth sensitvity.

### Compute the exact query response

#### Generate a random graph

In [11]:
n = 2 ** 8
p = 0.1
g = nx.random_graphs.gnp_random_graph(n=n, p=p)

bound = 10.0  # An upper bound on the edge weights in the graph
weights = {e: bound * np.random.randint(1, 11) / 10.0 for e in g.edges()}
nx.set_edge_attributes(g, weights, "weight")

#### Compute the exact cost of a minimum spanning tree

In [12]:
mst = nx.minimum_spanning_tree(g)
mst_cost = np.array([mst.size(weight="weight")])

### Compute the smooth sensitivity of the query

As per Nissim et al, let $G = (V, E)$ be a graph with vertex set $V$ and edge set $E$. For every $S \subset V$ we can define a cut to be the partition $(S, V \setminus S)$. We say an edge $(i,j)$ crosses the cut $S$ when $i \in S$ and $j \in V \setminus S$. For a cut $S \subset V$, let $w_t(S)$ denote the weight of the $t$-lightest edge in the cut. In the paper, the authors show that
$$ A^{(k)}(G) = \max \left(\max_{S \subset V} w_{k+1}(S), \max_{S \subset V} (w_{k+2}(S) - w_1(S))\right).$$
Furthermore, the authors show that if $T$ is a minimum spanning tree of $G$ then
$$ \max_{S \subset V} \left(w_{k+2}(S) - w_1(S))\right) = \max_{e \in T}\left(\max_{e-\text{cuts} \ S} w_{k+2}(S) - w_1(S)\right)$$
where an $(i,j)$-cut is a cut $S$ such that $(i,j)$ crosses $S$.

The authors then describe an efficient method for computing $\max_S w_k(S)$. We implement this algorithm with the function `find_max_cut_weight` below. We use the same algorithm to compute the max cut weight over $e$-cuts. This allows us to efficiently compute $A^{(k)}$ for all $k$ and thereby compute
$$S_{f,\epsilon}^*(G) = \max_{0 \leq k \leq n} e^{-\epsilon k} A^{(k)}(G).$$

In [13]:
def retrieve_min_cut_cost(costs, key, g, w, **args):
    """ Return the minimum cut cost for an unweighted graph related to g.
    
        If a key is in costs, then return costs[key]
        Otherwise, generate the appropriate value for costs[key],
        add this new (key, value pair) to costs and then
        return costs[key].
    """
    if key not in costs:
        gw = nx.Graph()
        gw.add_nodes_from(g)
        gw.add_edges_from([e for e in g.edges() if g.edges[e]["weight"] <= w[key]])
        costs[key] = nx.edge_connectivity(gw, **args)
    return costs[key]

In [14]:
def find_max_cut_weight(k, w, bound, costs, **kwargs):
    """
    Compute max_{e-cuts S} w_k(S).
    If kwargs is none then this is the same as max_{S \subset V} w_k(S)
    
    Parameters:
        k: The index of the weight of interest
        w: A list of sorted edge-weights
        bound: An upper bound for the edge-weights in w
        costs: A dictionary of min_cut_costs
        
        kwargs (optional): Two nodes that define an edge e which
                           must be separated by any cut under consideration.
            s: a source node
            t: a destination node
    
    Returns:
        max_{e-cuts S} w_k(S)
    """
    cost = retrieve_min_cut_cost(costs, len(w) - 1, g, w, **kwargs)
    if cost <= k:
        return bound

    cost = retrieve_min_cut_cost(costs, 0, g, w, **kwargs)
    if cost > k:
        return w[0]

    high = len(w) - 1
    low = 0
    while (high - low) > 1:
        mid = (low + high) // 2
        cost = retrieve_min_cut_cost(costs, mid, g, w, **kwargs)
        if cost <= k:
            low = mid
        else:
            high = mid
    return w[high]

#### Compute $\max_{S \subset V} w_{k+1}(S)$ for all $0 \leq k \leq n$.

In [15]:
edge_weights = [g.edges[e]["weight"] for e in g.edges()]
edge_weights = sorted(set(edge_weights))
costs = dict()
# Notice here that we use k instead of k+1 to correct for 0-up indexing in python
lsd1 = np.array([find_max_cut_weight(k, edge_weights, bound, costs) for k in range(n + 1)])

#### Compute $\max_{e \in T} \left(\max_{e-\text{cuts} \ S} \left(w_{k+2} - w_1(S)\right)\right)$ for all $0 \leq k \leq n$

In [ ]:
mst = nx.minimum_spanning_tree(g)
lsd2 = np.zeros(n + 1)
for e in mst.edges():
    costs = dict()
    # Notice here that we use k+1 instead of k+2 to correct for 0-up indexing in python
    max_cut_weights = [
        find_max_cut_weight(k + 1, edge_weights, bound, costs, s=e[0], t=e[1])
        for k in range(n + 1)
    ]
    lsd2_e = np.array(max_cut_weights) - mst.edges[e]["weight"]
    lsd2 = np.maximum(lsd2, lsd2_e)

#### Compute smooth sensitivity

In [ ]:
# Compute the local sensitivity at distance s for 0 <= s <= n
lsd = np.maximum(lsd1, lsd2)

# Compute the smooth sensitivity
epsilon = 1.0
smooth_scaling = np.exp(-epsilon * np.arange(n + 1))
smooth_sensitivity = np.max(lsd * smooth_scaling)

### Add noise scaled according to the smooth sensitvity
We create a differentially private release mechanism by adding noise from the Cauchy distribution scaled according to the smooth sensitivity. Because the Cauchy distributed random variables are real-valued, the differentially private query response will be real-valued despite the exact query response being integer-valued.

In [ ]:
# Create a differentiall private release mechanism
mechanism = CauchyMechanism(epsilon=epsilon)

# Compute the differentially private query response
dp_mst_cost = mechanism.release(mst_cost, smooth_sensitivity)

#### Display results

In [ ]:
print("Exact MST cost = %f" % mst_cost)
print("Differentially private MST cost = %f" % dp_mst_cost)